# MarsToday: evaluating Mars climate through REMS sensor onboard Curiosity Mars rover

## Import box

In [1]:
#Imports
import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv
import os
from pandas import json_normalize
import re
from matplotlib import pyplot as plt
from IPython.display import Image
from IPython.core.display import HTML

In [2]:
load_dotenv()

True

## Data first visualization

In [3]:
weather = pd.read_csv("../data/mars-weather.csv")
weather

,id,terrestrial_date,sol,ls,month,min_temp,max_temp,pressure,wind_speed,atmo_opacity
0,1895,2018-02-27,1977,135,Month 5,-77.0,-10.0,727.0,NaN,Sunny
1,1893,2018-02-26,1976,135,Month 5,-77.0,-10.0,728.0,NaN,Sunny
2,1894,2018-02-25,1975,134,Month 5,-76.0,-16.0,729.0,NaN,Sunny
3,1892,2018-02-24,1974,134,Month 5,-77.0,-13.0,729.0,NaN,Sunny
4,1889,2018-02-23,1973,133,Month 5,-78.0,-18.0,730.0,NaN,Sunny
...,...,...,...,...,...,...,...,...,...,...
1889,24,2012-08-18,12,156,Month 6,-76.0,-18.0,741.0,NaN,Sunny
1890,13,2012-08-17,11,156,Month 6,-76.0,-11.0,740.0,NaN,Sunny
1891,2,2012-08-16,10,155,Month 6,-75.0,-16.0,739.0,NaN,Sunny
1892,232,2012-08-15,9,155,Month 6,NaN,NaN,NaN,NaN,Sunny


## Connection with NASA API

In [4]:
# This is a function to call NASA api with a given value of Sol day and Camera type.

def call_Curiosity (date, camera):
    """
    This is a function that calls NASA API 'Mars Rover Photos' with two arguments. It returns the url from
    a specific camera onboard Curiosity rover.
    date: input the desired date in the format YYYY-MM-DD as a STRING,
    camera: select between FHAZ, RHAZ, MAST, CHEMCAM, MAHLI, MARDI, NAVCAM, PANCAM, MINITES, as STRING
    
    """
        
    try:
        nasa = os.getenv("token")
        url = f"https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?earth_date={date}&camera={camera}&api_key={nasa}"
        request = requests.get(url)
        df = pd.DataFrame(request.json())
        df_clean = pd.DataFrame(df.values[0][0])
        image_url = list(df_clean["img_src"])[0]

        return f"Image available for camera {camera} onboard Curiosity rover: {image_url}"

    except:
            
        return f"No image available on {date} for camera {camera} onboard Curiosity rover, please select another Sol day"    

## Getting Mars pictures

In [5]:
def get_pictures_Curiosity(date):
    
    """
    This is a function that calls call_NASA function with one argument. It returns the url of all the pictures 
    taken by all the cameras of Curiosity rover from a specific Sol date.
    date: input the desired date in the format YYYY-MM-DD as STRING.
    
    """       
    cameralist = ["FHAZ", "RHAZ", "MAST", "CHEMCAM", "MAHLI", "MARDI", "NAVCAM", "PANCAM", "MINITES"]
    for i in cameralist:
        print(call_Curiosity(date, i))
    pass

# Data cleaning

In [6]:
# Function to remove undesired columns

def remove_columns(df, column_name):
    
    df.drop(columns=f"{column_name}", inplace=True)
    
    return df

In [7]:
remove_columns(weather, "wind_speed")
remove_columns(weather, "id")
remove_columns(weather, "ls")

,terrestrial_date,sol,month,min_temp,max_temp,pressure,atmo_opacity
0,2018-02-27,1977,Month 5,-77.0,-10.0,727.0,Sunny
1,2018-02-26,1976,Month 5,-77.0,-10.0,728.0,Sunny
2,2018-02-25,1975,Month 5,-76.0,-16.0,729.0,Sunny
3,2018-02-24,1974,Month 5,-77.0,-13.0,729.0,Sunny
4,2018-02-23,1973,Month 5,-78.0,-18.0,730.0,Sunny
...,...,...,...,...,...,...,...
1889,2012-08-18,12,Month 6,-76.0,-18.0,741.0,Sunny
1890,2012-08-17,11,Month 6,-76.0,-11.0,740.0,Sunny
1891,2012-08-16,10,Month 6,-75.0,-16.0,739.0,Sunny
1892,2012-08-15,9,Month 6,NaN,NaN,NaN,Sunny


In [8]:
# Created a new column
weather["Mean_temp (°C)"] = ((weather["min_temp"] + weather["max_temp"])/2)

In [9]:
# Clean atmosphere column

def clean_atmosphere(df,column_name,string,replacement):
    df[f"{column_name}"] = df[f"{column_name}"].replace(f"{string}",f"{replacement}")    
    return df

In [10]:
clean_atmosphere(weather,"atmo_opacity","Sunny","1")
clean_atmosphere(weather,"atmo_opacity","--","0")

,terrestrial_date,sol,month,min_temp,max_temp,pressure,atmo_opacity,Mean_temp (°C)
0,2018-02-27,1977,Month 5,-77.0,-10.0,727.0,1,-43.5
1,2018-02-26,1976,Month 5,-77.0,-10.0,728.0,1,-43.5
2,2018-02-25,1975,Month 5,-76.0,-16.0,729.0,1,-46.0
3,2018-02-24,1974,Month 5,-77.0,-13.0,729.0,1,-45.0
4,2018-02-23,1973,Month 5,-78.0,-18.0,730.0,1,-48.0
...,...,...,...,...,...,...,...,...
1889,2012-08-18,12,Month 6,-76.0,-18.0,741.0,1,-47.0
1890,2012-08-17,11,Month 6,-76.0,-11.0,740.0,1,-43.5
1891,2012-08-16,10,Month 6,-75.0,-16.0,739.0,1,-45.5
1892,2012-08-15,9,Month 6,NaN,NaN,NaN,1,NaN


In [11]:
# Function to rename columns

def rename_columns(df, old_name, new_name):
    df.rename(columns={f"{old_name}": f"{new_name}"}, inplace=True)
    return df

In [12]:
# Rename columns
rename_columns(weather, "terrestrial_date", "Earth Date")
rename_columns(weather, "sol", "Sol")
rename_columns(weather, "month", "Month")
rename_columns(weather, "month", "Month")
rename_columns(weather, "min_temp", "Min_temp (°C)")
rename_columns(weather, "max_temp", "Max_temp (°C)")
rename_columns(weather, "pressure", "Pressure (Pa)")
rename_columns(weather, "atmo_opacity", "Atmo_opacity")

,Earth Date,Sol,Month,Min_temp (°C),Max_temp (°C),Pressure (Pa),Atmo_opacity,Mean_temp (°C)
0,2018-02-27,1977,Month 5,-77.0,-10.0,727.0,1,-43.5
1,2018-02-26,1976,Month 5,-77.0,-10.0,728.0,1,-43.5
2,2018-02-25,1975,Month 5,-76.0,-16.0,729.0,1,-46.0
3,2018-02-24,1974,Month 5,-77.0,-13.0,729.0,1,-45.0
4,2018-02-23,1973,Month 5,-78.0,-18.0,730.0,1,-48.0
...,...,...,...,...,...,...,...,...
1889,2012-08-18,12,Month 6,-76.0,-18.0,741.0,1,-47.0
1890,2012-08-17,11,Month 6,-76.0,-11.0,740.0,1,-43.5
1891,2012-08-16,10,Month 6,-75.0,-16.0,739.0,1,-45.5
1892,2012-08-15,9,Month 6,NaN,NaN,NaN,1,NaN


In [13]:
# Function to clean the month column

def clean_month(df,column_name):
    df[f"{column_name}"] = df[f"{column_name}"].str.extract(r"(\d)")    
    return df

In [14]:
clean_month(weather, "Month")

,Earth Date,Sol,Month,Min_temp (°C),Max_temp (°C),Pressure (Pa),Atmo_opacity,Mean_temp (°C)
0,2018-02-27,1977,5,-77.0,-10.0,727.0,1,-43.5
1,2018-02-26,1976,5,-77.0,-10.0,728.0,1,-43.5
2,2018-02-25,1975,5,-76.0,-16.0,729.0,1,-46.0
3,2018-02-24,1974,5,-77.0,-13.0,729.0,1,-45.0
4,2018-02-23,1973,5,-78.0,-18.0,730.0,1,-48.0
...,...,...,...,...,...,...,...,...
1889,2012-08-18,12,6,-76.0,-18.0,741.0,1,-47.0
1890,2012-08-17,11,6,-76.0,-11.0,740.0,1,-43.5
1891,2012-08-16,10,6,-75.0,-16.0,739.0,1,-45.5
1892,2012-08-15,9,6,NaN,NaN,NaN,1,NaN


In [15]:
#Create the Season column

weather["Season"] = weather["Month"]

for i in range(len(weather["Season"])):

    if i in range(1,4):
        
        weather["Season"] = weather["Season"].replace(f"{i}","Winter")
    
    elif i in range(4,7):

        weather["Season"] = weather["Season"].replace(f"{i}","Spring")
    
    elif i in range(7,10):

        weather["Season"] = weather["Season"].replace(f"{i}","Summer")
    
    elif i in range(10,13):

        weather["Season"] = weather["Season"].replace(f"{i}","Autumn")

In [16]:
weather

,Earth Date,Sol,Month,Min_temp (°C),Max_temp (°C),Pressure (Pa),Atmo_opacity,Mean_temp (°C),Season
0,2018-02-27,1977,5,-77.0,-10.0,727.0,1,-43.5,Spring
1,2018-02-26,1976,5,-77.0,-10.0,728.0,1,-43.5,Spring
2,2018-02-25,1975,5,-76.0,-16.0,729.0,1,-46.0,Spring
3,2018-02-24,1974,5,-77.0,-13.0,729.0,1,-45.0,Spring
4,2018-02-23,1973,5,-78.0,-18.0,730.0,1,-48.0,Spring
...,...,...,...,...,...,...,...,...,...
1889,2012-08-18,12,6,-76.0,-18.0,741.0,1,-47.0,Spring
1890,2012-08-17,11,6,-76.0,-11.0,740.0,1,-43.5,Spring
1891,2012-08-16,10,6,-75.0,-16.0,739.0,1,-45.5,Spring
1892,2012-08-15,9,6,NaN,NaN,NaN,1,NaN,Spring


In [18]:
weather.to_csv("../data/mars-weather-cleaned.csv", index = False)